## cross-entropy
### cross-entropy 方法是 policy-based，由非線性函數(NN)來產生policy

### cross-entropy的核心是丟棄不好的Episode，並訓練出更好的Episode
### step :
#### 1.使用我們現有的模型和環境執行N個Episode
#### 2.計算每個Episode的total reward並決定reward boundary。通常使用total reward的百分數當做reward boundary。
#### 3.丟棄total reward低於reward boundary的Episode。
#### 4.使用observation當作input，訓練剩餘的"精英Episode"，並已選用的action當作輸出。
#### 5.回到step1，直到我們對輸出結果滿意為止。

In [1]:
import gym
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

### 模型的核心是個單hidden layer的NN，具有ReLU和128個"單hidden layer神經元"。
### 其他hyper param. 是隨機設定並不做校正。

### hidden layer的神經元數量,每次迭代的Episode數量，過濾"精英Episode"的百分數

In [2]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [3]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

## 定義以下helper class，他們用標準套件collections中的namedtuple類別來產生named tuple(具名常數串列)

In [4]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [5]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [6]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

### start training

In [8]:
env = gym.make("CartPole-v0")
env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-cartpole")

for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
        iter_no, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    if reward_m > 199:
        print("Solved!")
        break
writer.close()

0: loss=0.688, reward_mean=20.5, reward_bound=25.5
1: loss=0.668, reward_mean=29.4, reward_bound=31.5
2: loss=0.654, reward_mean=33.6, reward_bound=42.5
3: loss=0.654, reward_mean=43.2, reward_bound=50.0
4: loss=0.650, reward_mean=37.9, reward_bound=42.0
5: loss=0.647, reward_mean=45.8, reward_bound=47.0
6: loss=0.635, reward_mean=48.6, reward_bound=61.5
7: loss=0.629, reward_mean=53.7, reward_bound=61.5
8: loss=0.619, reward_mean=52.8, reward_bound=66.5
9: loss=0.612, reward_mean=45.1, reward_bound=53.0
10: loss=0.602, reward_mean=59.9, reward_bound=69.5
11: loss=0.609, reward_mean=67.6, reward_bound=70.0
12: loss=0.587, reward_mean=85.4, reward_bound=96.0
13: loss=0.598, reward_mean=66.2, reward_bound=65.0
14: loss=0.591, reward_mean=77.8, reward_bound=97.0
15: loss=0.584, reward_mean=104.8, reward_bound=123.0
16: loss=0.569, reward_mean=86.1, reward_bound=115.0
17: loss=0.585, reward_mean=102.2, reward_bound=120.0
18: loss=0.582, reward_mean=89.4, reward_bound=107.0
19: loss=0.577, 